## 1. import modules

In [15]:
from PIL import Image,ImageFilter
from scipy.misc import toimage
from operator import itemgetter
# from skimage import measure
import numpy as np
import copy
import heapq
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage.filters import median_filter


In [16]:
image_path = 'Screen Shot 2019-06-30 at 10.35.38 PM.png'

In [10]:
im = Image.open(image_path)

In [18]:
gray = cv2.cvtColor(cv2.UMat(image_path), cv2.COLOR_RGB2GRAY)


TypeError: Required argument 'ranges' (pos 2) not found